In [5]:
import argparse
import logging
import json
import numpy as np
import os
import trimesh
from pathos.multiprocessing import ProcessPool as Pool
from scipy.spatial import cKDTree as KDTree
from scipy.optimize import linear_sum_assignment


In [6]:
#From DeepSDF metrics 
# https://github.com/facebookresearch/DeepSDF/tree/main/deep_sdf/metrics
# 

def compute_trimesh_chamfer(gt_points, gen_mesh, offset, scale, num_mesh_samples=30000):
    """
    This function computes a symmetric chamfer distance, i.e. the sum of both chamfers.

    gt_points: trimesh.points.PointCloud of just poins, sampled from the surface (see
               compute_metrics.ply for more documentation)

    gen_mesh: trimesh.base.Trimesh of output mesh from whichever autoencoding reconstruction
              method (see compute_metrics.py for more)

    """

    gen_points_sampled = trimesh.sample.sample_surface(gen_mesh, num_mesh_samples)[0]
   # print(gen_points_sampled)
    gen_points_sampled = gen_points_sampled / scale - offset

    # only need numpy array of points
    # gt_points_np = gt_points.vertices
   # print(gt_points)
    gt_points_np = gt_points.vertices

    # one direction
    gen_points_kd_tree = KDTree(gen_points_sampled)
    one_distances, one_vertex_ids = gen_points_kd_tree.query(gt_points_np)
    gt_to_gen_chamfer = np.mean(np.square(one_distances))

    # other direction
    gt_points_kd_tree = KDTree(gt_points_np)
    two_distances, two_vertex_ids = gt_points_kd_tree.query(gen_points_sampled)
    gen_to_gt_chamfer = np.mean(np.square(two_distances))

    return gt_to_gen_chamfer + gen_to_gt_chamfer

In [7]:
def compute_trimesh_emd(gt_points, gen_mesh, offset, scale, num_mesh_samples=500):
    """
    This function computes a symmetric chamfer distance, i.e. the sum of both chamfers.

    gt_points: trimesh.points.PointCloud of just poins, sampled from the surface (see
               compute_metrics.ply for more documentation)

    gen_mesh: trimesh.base.Trimesh of output mesh from whichever autoencoding reconstruction
              method (see compute_metrics.py for more)

    """

    gen_points_sampled = trimesh.sample.sample_surface(gen_mesh, num_mesh_samples)[0]

    gen_points_sampled = gen_points_sampled / scale - offset

    # only need numpy array of points
    # gt_points_np = gt_points.vertices
    gt_points_np = gt_points.vertices
    gt_points_np = np.random.permutation(gt_points_np)[:num_mesh_samples]

    # hist0 = hist1 = np.ones([num_mesh_samples], dtype=np.float64) / num_mesh_samples
    dist = np.linalg.norm(np.expand_dims(gt_points_np, axis=0) - np.expand_dims(gen_points_sampled, axis=1), axis=-1)
    # dist = dist.astype(np.float64)
    # emd = pyemd.emd(hist0, hist1, dist)
    assignment = linear_sum_assignment(dist)
    emd = dist[assignment].sum() / num_mesh_samples

    return emd


In [8]:
DIRECTORY_MODELS = '../reconstruction/mesh' # Mesh Directory
DIRECTORY_GT = '../reconstruction/mesh'
MODEL_EXTENSION = '.ply'
def get_model_files():
    for directory, _, files in os.walk(DIRECTORY_MODELS):
        for filename in files:
            if filename.endswith(MODEL_EXTENSION):
                yield  os.path.join(filename)
    
               
files = list(get_model_files())
print("the number of Dataset:", len(files))

the number of Dataset: 4


In [10]:
cd_mean = []
emd_mean = []

for ii, data in enumerate(files):
    data_dir = os.path.join(DIRECTORY_MODELS, data)
    rec_data = trimesh.load(data_dir)
#    print(data_dir)
    gt_dir = os.path.join(DIRECTORY_GT, data)
    gt_data = trimesh.load(gt_dir)
    
    Chamfer = compute_trimesh_chamfer(rec_data,gt_data,0, 1)
    EMD = compute_trimesh_emd(rec_data,gt_data,0, 1)
    cd_mean.append(Chamfer)
    emd_mean.append(EMD)
CD_M = sum(cd_mean)/len(cd_mean)
EMD_M = sum(emd_mean)/len(emd_mean)
print("Chamfer: ",CD_M)
print("EMD: ", EMD_M)

Chamfer:  2.7795624681639686e-05
EMD:  0.06760279880444281
